In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Rescaling, RandomFlip, RandomRotation
from tensorflow.keras.models import Model
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt

2024-07-08 11:39:17.777895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 11:39:17.777993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 11:39:17.868974: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
#Creating a training data directory and dividing the dataset into training set and validation set and reshaping the images
train_data_dir = '/kaggle/input/indian-food-images-dataset/Indian Food Images/Indian Food Images/'
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    image_size=(224, 224),  # Resize images to 224x224
    batch_size=32,         # Batch size
    label_mode='categorical',  # Categorical labels for multi-class classification
    validation_split=0.2,  # Split the data for validation (80% train, 20% validation)
    subset='training',     # Specify subset for training
    seed=123               # Random seed for reproducibility
)


# Create the validation dataset
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    image_size=(224, 224),  # Resize images to 224x224
    batch_size=32,          # Batch size
    label_mode='categorical',  # Categorical labels for multi-class classification
    validation_split=0.2,   # Split the data for validation (80% train, 20% validation)
    subset='validation',    # Specify subset for validation
    seed=123                # Random seed for reproducibility
)

Found 4000 files belonging to 80 classes.
Using 3200 files for training.
Found 4000 files belonging to 80 classes.
Using 800 files for validation.


In [4]:
# Print the shapes of the datasets to verify
for images, labels in train_dataset.take(1):
    print(f'Train images shape: {images.shape}')
    print(f'Train labels shape: {labels.shape}')

for images, labels in validation_dataset.take(1):
    print(f'Validation images shape: {images.shape}')
    print(f'Validation labels shape: {labels.shape}')
    

Train images shape: (32, 224, 224, 3)
Train labels shape: (32, 80)
Validation images shape: (32, 224, 224, 3)
Validation labels shape: (32, 80)


In [5]:
# Data augmentation to increase the generalization of the model and decrease the bias
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

In [6]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [7]:
IMG_SHAPE = (224,224,3)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = tf.keras.applications.resnet50.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy')])

model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ sequential[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ sequential[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 2048)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 80)        │    163,920 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,751,632 (90.61 MB)

 Trainable params: 163,920 (640.31 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
initial_epochs = 20
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 30s 169ms/step - accuracy: 0.0142 - loss: 5.6767 - val_accuracy: 0.0288 - val_loss: 4.4424
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 34s 153ms/step - accuracy: 0.0295 - loss: 4.9512 - val_accuracy: 0.0712 - val_loss: 4.0941
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 153ms/step - accuracy: 0.0474 - loss: 4.5808 - val_accuracy: 0.1250 - val_loss: 3.8105
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 155ms/step - accuracy: 0.0791 - loss: 4.2648 - val_accuracy: 0.1525 - val_loss: 3.5756
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 157ms/step - accuracy: 0.0865 - loss: 4.0723 - val_accuracy: 0.1863 - val_loss: 3.3806
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 160ms/step - accuracy: 0.1229 - loss: 3.7616 - val_accuracy: 0.1975 - val_loss: 3.2109
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 160ms/step - accuracy: 0.1641 - loss: 3.6151 - val_accuracy: 0.2313 - val_loss: 3.0576
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 159ms/step - accuracy: 0.1714 - loss: 3